# Maven Power Outage Challenge

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium 
import openpyxl

In [22]:
# disturbances = pd.read_excel()

disturbances = pd.read_excel('DOE_Electric_Disturbance_Events.xlsx', engine='openpyxl')
disturbances

,Table B.2.,"Major Disturbances and Unusual Occurrences, 2002",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date,NERC Region,Time,Area,Type of Disturbance,Loss (megawatts),Number of Customers Affected,Restoration Time
2,January,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002-01-30 00:00:00,SPP,06:00:00,Oklahoma,Ice Storm,500,1881134,2002-02-07 12:00:00
4,None,NaN,None,NaN,NaN,NaN,NaN,None
5,2002-01-29 00:00:00,SPP,Evening,Metropolitan Kansas City Area,Ice Storm,500-600,270000,NaN
6,2002-01-30 00:00:00,SPP,16:00:00,Missouri,Ice Storm,210,95000,2002-02-10 21:00:00
7,February,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2002-02-27 00:00:00,WSCC,10:48:00,California,Interruption of Firm Load,300,255000,2002-02-27 11:35:00
9,March,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
disturbances.columns

Index(['Table B.2.', 'Major Disturbances and Unusual Occurrences, 2002',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7'],
      dtype='object')

Initial notes about data:
1. the column headers seem to be located in row 1
2. date formats are inconsistent
3. I will need to check what NERC Region means
4. What does area refer to? Sometimes it is an entire state while other times it is a more specific part of a state.
5. lots of NaN that needs to be addressed
6. some columns have inconsistent data located in them - perhaps these rows need to be removed

In [21]:
#What is the shape of the data:

disturbances.shape

(41, 8)